La libreria empleada para construir redes neuronales que emplearemos sera PyTorch.
+ https://pytorch.org/

In [1]:
import torch
import torchvision

import matplotlib.pyplot as plt
import numpy as np

## Datasets
El dataset empleado en este laboratorio es el CIFAR-10, el cual contiene 60000 imagenes a color de 32x32 pixeles ordenadas en 10 classes, con 6000 imagenes por clase.

Pytorch ya incorpora este dataset. Todos los datasets incluidos en la libreria se pueden ver en el siguiente enlace:
+ https://pytorch.org/vision/stable/datasets.html

In [2]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
import torchvision.datasets as datasets

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Explora el dataset cargado:
+ ¿Que tipo de dato es?
+ ¿Cuales son sus atributos?
+ ¿Que contiene el primer elemento?
+ Muestra el primer elemento como imagen

In [ ]:
type(dataset)

In [ ]:
 # llamar a la variable que almacena el conjuntos de datos

In [ ]:
dataset[][0].shape # con el metodo .shape revisa las dimensiones de las imagenes

In [ ]:
img = dataset[][0].numpy() / 2 + 0.5 # quita la normalizacion de la imagen deseada para mostrar por pantalla
plt.imshow(np.transpose(img, (1, 2, 0)))
plt.show()

### Reducir cantidad de datos
En la demostracion en clases por motivos de tiempo reduciremos la cantidad de datos por categoria. En caso de querer reproducir con el dataset completo no correr estas celdas.

In [8]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

targets = np.array(dataset.targets)
num_classes = 10 # Número de clases en CIFAR-10

# Número de ejemplos que se quiere por clase (puedes ajustarlo)
samples_per_class = 100

# Proporción del conjunto de validación (por ejemplo, 20% de los datos)
validation_split = 0.2

# Crear listas para los índices de entrenamiento y validación
train_indices = []
val_indices = []

# Iterar sobre cada clase y dividir los índices en entrenamiento y validación
for class_idx in range(num_classes):
    # Obtener los índices de todos los ejemplos de la clase actual
    class_indices = np.where(targets == class_idx)[0]

    # Dividir los índices de la clase en entrenamiento y validación
    train_idx, val_idx = train_test_split(class_indices, test_size=validation_split, random_state=42)

    # Añadir los índices a las respectivas listas
    train_indices.extend(train_idx)
    val_indices.extend(val_idx)

# Crear los subconjuntos de entrenamiento y validación
train_subset = Subset(dataset, train_indices)
val_subset = Subset(dataset, val_indices)

### Dataloaders

Los Dataloaders nos ayudan a particionar cada dataset en bloques mas pequeños los cuales seran pasados durante el entrenamiento, estos se denominan **batch o mini-batch**.

Un argumento relevante ademas del tamaño de cada **batch**, es si rebarajar o reordenar (shuffle) los datos, esto puede ayudar a reducir el sobreajuste (u overfitting).

In [9]:
batch_size =  # Elegir numero de muestras por batch

trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=True)

## Red neuronal Convolucional - CNN

Las CNN incorporan al menos 3 tipos de capas nuevas que trabajan con las imagenes:
+ Convolucionales: Realizan convoluciones a las imagenes empleando un kernel de tamaño configurable
+ Pooling: Realizan reduccion de dimensionalidad, aplicando promedios (Average Pooling), seleccionando los valores maximos (Max Pooling), entre otros
+ Flatten: Toma los tensores y los aplana para poder se procesados por capas densas o lineales tradicionales

En el siguiente bloque podras ver el resultado de una operacion de convolucion y de pooling.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

imagen_demo = train_subset[][0] # Seleccionar imagen
conv_demo = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 3)
conv_image = conv_demo(imagen_demo)
print(conv_image.shape)

pool_demo = nn.MaxPool2d(2,2)
pool_image = pool_demo(conv_image)
print(pool_image.shape)

img = imagen_demo.permute(1,2,0).detach().numpy() / 2 + 0.5
conv_img = conv_image[0].detach().numpy()
pool_img = pool_image[0].detach().numpy()

fig, axs = plt.subplots(1,3)
axs[0].imshow(img)
axs[1].imshow(conv_img)
axs[2].imshow(pool_img)

plt.show()

### Creacion de CNN
Para hacer una red neuronal en Pytorch es necesario crear un modulo, para esto se define una clase la cual heredara de nn.Module.
Ademas para aplicar funciones de activacion a cada capa es necesario emplear el objeto functional.

In [30]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module): # Para rellenar
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 3) # seleccionar kernel
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)  # (16 - tamaño_kernel)/stride + 1 -> 11px
                                          # A realizar pooling se debe dividir el canal en 2 y truncar la parte entera, 11/2->5
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # n_canales * alto_imagen * ancho_imagen
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

cnn = CNN()

## Seleccion de Optimizador y Funcion de Costo
Debido a que la tarea seleccionada es la de clasificar a partir de multiples categorias, la funcion de perdida a utilizar es la entropia cruzada (Cross Entropy). La Funcion de costo sera la de Gradiente descendente estocastico.


In [31]:
import torch.optim as optim

criterion =  # Que funcion de perdida o criterio emplearias para este problema, revisar documentacion
optimizer =  # Que optimizador o metodo emplearias, revisar documentacion

## Entrenamiento
Utilizando la red creada, el optimizador y la funcion de costo, procederemos a entrenar nuestra red, este procedimiento es iterativo y estara limitado por la cantidad de epocas que seleccionemos.

Los pasos seguidos de forma general seran:
+ Definir la cantidad de epocas a entrenar
+ En un bucle se iteran los datos dentro del DataLoader de entrenamiento
+ Se realiza el paso hacia adelante (o forward) obteniendo la prediccion para el batch
+ Se computa el error obteniendo la funcion de perdida y se retropropaga el error en el paso hacia atras (o backward)
+ Se actualizan los pesos mediante el optimizador en nuestro caso el gradiente descendente
+ Se muestran, almacenan y/o calculan las metricas deseadas

In [ ]:
EPOCHS = 5
losses = []
for epoch in range(EPOCHS):
    epoch_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = cnn(inputs)
        # backward
        loss = criterion(outputs, labels)
        loss.backward()
        # optimizacion
        optimizer.step()

        # print statistics
        epoch_loss += loss.item()

    average_loss = epoch_loss / len(trainloader)  # Calculate average loss for the epoch
    losses.append(average_loss)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {average_loss:.4f}")

print('Entrenamiento finalizado')

In [ ]:
plt.plot(losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

El codigo puede ser mejorado obteniendo metricas con un conjunto de validacion, esto con el fin de analizar el proceso de aprendizaje, generalizacion, underfitting y/o overfitting.

Para realizar calculos con el modelo, por defecto se emplea el modo train(), para evitar modificar el modelo emplearemos el modo eval(), ademas para evitar el calculo de gradientes se desactiva el modulo autograd() el cual cumple esta funcion.

In [ ]:
EPOCHS = 5
train_losses = []
val_losses = []
for epoch in range(EPOCHS):
    train_loss = 0.0
    cnn.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # setea los gradientes en 0
        optimizer.zero_grad()

        # forward
        outputs = cnn(inputs)
        # backward
        loss = criterion(outputs, labels)
        loss.backward()
        # optimizacion
        optimizer.step()

        # print statistics
        train_loss += loss.item()

    average_train_loss = train_loss / len(trainloader)
    train_losses.append(average_train_loss)

    val_loss = 0.0
    cnn.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = cnn(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    average_validation_loss = val_loss / len(testloader)
    val_losses.append(average_validation_loss)

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {average_train_loss:.4f}, Validation Loss: {average_validation_loss:.4f}")

print('Entrenamiento finalizado')

Grafique ambas funciones de perdida utilizando matplotlib y comente sus resultados.

Guardado de modelo:

Los pesos de los modelos entrenados se pueden almacenar mediante la funcion torch.save()

In [16]:
torch.save(cnn.state_dict(), './cnn_cifar10.pth')

Carga de modelo:

Los pesos almacenados pueden ser cargados junto a la definicion del modelo(clase realizada previamente)

In [ ]:
cnn_loaded = CNN()
cnn_loaded.load_state_dict(torch.load('./cnn_cifar10.pth', weights_only=True))

Prueba de modelo:

Con el modelo cargado puedes seleccionar un dato del conjunto de validacion y realizar una prediccion.

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

dataiter = iter(testloader)

images, labels = next(dataiter)

imgs = torchvision.utils.make_grid(images)
img = imgs / 2 + 0.5
plt.imshow(np.transpose(img, (1, 2, 0)))
plt.show()

outputs = cnn_loaded(images)
_, predicted = torch.max(outputs, 1)

print('Predicho: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))
print('Verdadero: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))